In [1]:
# !pip install tqdm
# !pip install matplotlib
# !pip install seaborn

%load_ext autoreload

%autoreload 2

In [2]:
import sys
import os
#make the self written stuff findeable independent on who downloads it
#sts.path.append( ) makes the module importable
# os.path.dirname(os.getcwd()) provides the parent directory
sys.path.append(os.path.dirname(os.getcwd()))

In [3]:
import os
 
dirpath = os.getcwd()
print("current directory is : " + dirpath)
foldername = os.path.basename(dirpath)
print("Directory name is : " + foldername)


current directory is : C:\Users\Utis\Documents\GitHub\tensorflow_1.X_playground\regression\notebooks
Directory name is : notebooks


In [4]:
#sys.path

In [5]:
from data.generators import ToyDataset


In [6]:
from helpers import _TFColor, colors, lazy_property
import helpers
TFColor = _TFColor()

In [7]:
import numpy as np

import matplotlib.pyplot as plt

matplotlib_style = 'fivethirtyeight' #@param ['fivethirtyeight', 'bmh', 'ggplot', 'seaborn', 'default', 'Solarize_Light2', 'classic', 'dark_background', 'seaborn-colorblind', 'seaborn-notebook']
import matplotlib.pyplot as plt; plt.style.use(matplotlib_style)
import matplotlib.axes as axes;
from matplotlib.patches import Ellipse
%matplotlib inline
import seaborn as sns; sns.set_context('notebook')
from IPython.core.pylabtools import figsize
#@markdown This sets the resolution of the plot outputs (`retina` is the highest resolution)
notebook_screen_res = 'retina' #@param ['retina', 'png', 'jpeg', 'svg', 'pdf']
%config InlineBackend.figure_format = notebook_screen_res

In [8]:
figsize(20,7)

In [9]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
 
 
from tensorflow.python.platform import tf_logging as logging
 
logging.set_verbosity(logging.INFO)
logging.log(logging.INFO, "Tensorflow version " + tf.__version__)

INFO:tensorflow:Tensorflow version 1.13.1


In [10]:
from data.generators import ToyDataset

In [11]:
DATA_SEQ_LEN = 300#240


dataset = ToyDataset(DATA_SEQ_LEN,DATA_SEQ_LEN)
X, y = dataset.X, dataset.y

In [12]:
gdef = dataset.graph()

Now the shufflebuffer includes the whole dataset and we have loads of randomness in the sequence to work with.

In [13]:
# with tf.Session() as sess:
#     X_meh, y_meh = sess.run(it.get_next())
    
# plt.plot(X_meh,y_meh,'*')

In [14]:
# with tf.Graph().as_default() as g_1:
#   dataset = tf.data.Dataset.from_tensor_slices((X, y))
#   dataset = dataset.repeat()
#   dataset = dataset.shuffle(300)
#   dataset = dataset.batch(10)
#   it = dataset.make_one_shot_iterator()
#   get_next = tf.identity(it.get_next(), name = 'next')
#   X_ =  tf.identity(it.get_next()[0], name="X")
#   y_ =  tf.identity(it.get_next()[1], name="y")

#   gdef = g_1.as_graph_def()
  
#   #g_1.get_tensor_by_name('X:0')
#   #g_1.get_tensor_by_name("y:0")

#   #it = dataset.make_one_shot_iterator()

# #X,y = it.get_next()

In [34]:
# g2 = tf.Graph()

# with g2.as_default():
#   dataset_graph = tf.import_graph_def(gdef, return_elements = ['X:0',"y:0", 'next'])
#   next_ = g2.get_tensor_by_name('import/next:0')
#   #X_ = g2.get_tensor_by_name('import/X:0')
#   #y_ = g2.get_tensor_by_name('import/y:0')
  
#   with tf.Session() as sess:
#     X_, y_ = sess.run(next_)
#     #X_restored = sess.run(X_)
#     #y_restored = sess.run(y_)
#     plt.plot(X_,y_,'*')
#     print(len(X_))
    
    
    
    
#     X_, y_ = sess.run(next_)
#     #X_restored = sess.run(X_)
#     #y_restored = sess.run(y_)
#     plt.plot(X_,y_,'*')
#     print(len(X_))
    
    
    
#     X_, y_ = sess.run(next_)
#     #X_restored = sess.run(X_)
#     #y_restored = sess.run(y_)
#     plt.plot(X_,y_,'*')
#     print(len(X_))
#     #plt.plot(y_,'*')
#     #plt.plot(y_restored)
#     #plt.plot(X_restored)
#     #plt.plot(sess.run(y_))
#     #plt.plot(X_restored)

#   print(type(dataset_graph))
#   print(g2)

In [35]:
#https://stackoverflow.com/questions/33748552/tensorflow-how-to-replace-a-node-in-a-calculation-graph

In [36]:
#from models.networks import EnsembleNetwork

In [37]:

class EnsembleNetwork(object):
    def __init__(
            self,
            ds_graph,
            num_neurons=[10, 10, 10],
            num_features=1,
            learning_rate=0.001,
            activations=None,  #[tf.nn.tanh,tf.nn.relu,tf.sigmoid]
            dropout_layers=None,  #[True,False,True]
            initialisation_scheme=None,  #[tf.random_normal,tf.random_normal,tf.random_normal]
            optimizer=None,  #defaults to GradiendDescentOptimizer,
            num_epochs=None,  #defaults to 1,
            seed=None,
            #adversarial=None
    ):

        #necessary parameters
        self.num_neurons = num_neurons
        self.num_layers = len(num_neurons)
        self.num_features = num_features
        self.learning_rate = learning_rate
        self.ds_graph = ds_graph
        #self.adversarial = adversarial or ADVERSARIAL
        
        


        #optional parameters
        self.optimizer = optimizer or tf.train.GradientDescentOptimizer
        self.activations = activations or [tf.nn.tanh] * self.num_layers
        self.initialisation_scheme = initialisation_scheme or tf.random_normal
        self.num_epochs = num_epochs or 1000
        self.seed = seed or None
        self.initialise_graph
        self.initialise_session
        
        


    @lazy_property
    def initialise_graph(self):
        #initialise graph
        self.g = tf.Graph()
        #build graph with self.graph as default so nodes get appended
        with self.g.as_default():
            dataset_graph = tf.import_graph_def(self.ds_graph, return_elements = ['X:0',"y:0"])
            self.next = self.g.get_tensor_by_name('import/next:0')
            #self.X = tf.cast(self.g.get_tensor_by_name('import/X:0'), dtype=tf.float32)
            #self.y = tf.cast(self.g.get_tensor_by_name('import/y:0'), dtype=tf.float32)
            self.init_network
            self.predict_graph
            self.error_graph
            self.train_graph
            self.init = tf.global_variables_initializer()

    @lazy_property
    def initialise_session(self):
        #initialise session
        self.session = tf.Session(graph=self.g)
        #initialise global variables
        self.session.run(self.init)

    @lazy_property
    def init_network(self):
        if self.seed:
            tf.set_random_seed(self.seed)
        #inputs
        #self.X = self.x_in#batch[0]#self.X#tf.placeholder(tf.float32, (None, self.num_features))
        #self.y = self.y_in#batch[1]#y#tf.placeholder(tf.float32, (None, 1))  #regression = 1
        self.X = self.next[0]
        self.y = self.next[1]

        #lists for storage
        self.w_list = []
        self.b_list = []

        #add input x first weights
        self.w_list.append(
            tf.Variable(
                self.initialisation_scheme(
                    [self.num_features, self.num_neurons[0]]),
                name='w_0'))  #first Matrix
        self.b_list.append(
            tf.Variable(
                tf.ones(shape=[self.num_features]), name='b_0' ))
        #for each layer over 0 add a n x m matrix and a bias term
        for i, num_neuron in enumerate(self.num_neurons[1:]):
            n_inputs = self.num_neurons[i]  #for first hidden layer 3
            n_outputs = self.num_neurons[i + 1]  #for first hidden layer 5

            self.w_list.append(
                tf.Variable(
                    self.initialisation_scheme([n_inputs, n_outputs]),
                    name='w_' + str(i)))
            self.b_list.append(
                tf.Variable(tf.ones(shape=[n_inputs]), name='b_' + str(i)))

        #add last layer m  x 1 for output
        self.w_list.append(
            tf.Variable(
                self.initialisation_scheme([self.num_neurons[-1], 1]),
                name='w_-1'))  #this is a regression
        self.b_list.append(
            tf.Variable(
                tf.ones(shape=[self.num_neurons[-1]]), name='b_' + str(
                    len(self.num_neurons) + 1)))

    @lazy_property
    def predict_graph(self):
        #set layer_input to input
        layer_input = self.X

        #for each layer do
        for i, w in enumerate(self.w_list):

            #z = input x Weights
            a = tf.matmul(layer_input, w, name='matmul_' + str(i))

            #z + bias
            if i <= self.num_layers:
                bias = self.b_list[i]
                a = tf.add(a, bias)

            #a = sigma(z) if not last layer and regression
            if i < self.num_layers:

                a = self.activations[i](a)
            #set layer input to a for next cycle
            layer_input = a

        return a

    @lazy_property
    def error_graph(self):

        #y_hat is a // output of prediction graph
        y_hat = self.predict_graph

        #error is mean squared error of placehilder y and prediction
        error = tf.losses.mean_squared_error(self.y, y_hat)  #tf.square(
        #self.y - y_hat)  #

        return error

    @lazy_property
    def train_graph(self):

        #error is the error from error graph
        error = self.error_graph

        #optimizer is self.optimizer
        optimizer = self.optimizer(learning_rate=self.learning_rate)
        
        
        return optimizer.minimize(error)
      
      
    def fit(self,num_epochs=10):
      error_list = []
      for i in range(num_epochs):
        error_list.append(self.session.run(self.train_graph))
        
      return error_list

      
    def fit_dataset(self,X,y):
      self.X = X
      self.y = y
      self.session.run(self.train_graph)
      
    def fit_generator(self,generator):
      self.session.run(self.train_graph, generator)

    def train_offline(self, X, y):
        for epoch in range(self.num_epochs):

            self.session.run(self.train_graph,
                             feed_dict={self.X: X,
                                        self.y: y})

    def train(self, X, y, shuffle=True, online=False):
        #print('X is {}'.format(X[:10]))
        if online:
            for epoch in range(self.num_epochs):
                self.train_one_epoch(X, y, shuffle)
        else:
            self.train_offline(X, y)

    def train_one_epoch(self, epoch_X, epoch_y, shuffle=True):
        if shuffle == True:
            epoch_X, epoch_y, _ = unison_shuffled_copies(
                np.squeeze(epoch_X), np.squeeze(epoch_y), True)
        #print(epoch_X[:10])

        for batch_X, batch_y in zip(epoch_X, epoch_y):
            self.train_one(batch_X, batch_y)
            #if self.adversarial:
            #    self.train_one(batch_X+0.05, batch_y)
            #    self.train_one(batch_X-0.05, batch_y)

    def train_one(self, batch_X, batch_y):
        batch_X = np.expand_dims(batch_X, 1)
        batch_y = np.expand_dims(batch_y, 1)
        self.session.run(self.train_graph,
                         feed_dict={self.X: batch_X,
                                    self.y: batch_y})

    def train_and_evaluate(self, X, y, shuffle=False):
        errors = []
        for epoch in range(self.num_epochs):

            #self.session.run(self.train_graph,
            #                 feed_dict={self.X: X,
            #                            self.y: y})
            self.train_one_epoch(X, y, shuffle)

                    

            errors += list(np.sqrt((y - self.predict(X))**2))

            #self.session.run(self.predict_graph,
            #                          feed_dict={self.X: X,
            #                                     self.y: y}))**2))

        return errors

    def predict(self, X):
        #return self.session.run(self.predict_graph)
        return self.session.run(self.predict_graph, feed_dict={self.X: X})
      
    def predict_meh(self):
      """never use this it's stupid"""
      return self.session.run(self.predict_graph) 

    def kill(self):
        self.session.close()


In [38]:


class EnsembleNetwork(object):
    def __init__(
            self,
            ds_graph,
            num_neurons=[10, 5, 3],
            num_features=1,
            learning_rate=0.001,
            activations=None,  #[tf.nn.tanh,tf.nn.relu,tf.sigmoid]
            dropout_layers=None,  #[True,False,True]
            initialisation_scheme=None,  #[tf.random_normal,tf.random_normal,tf.random_normal]
            optimizer=None,  #defaults to GradiendDescentOptimizer,
            num_epochs=None,  #defaults to 1,
            seed=None,
            adversarial=None,
            initialisation_params=None,
            l2=None,
            l=None):

        #necessary parameters
        self.num_neurons = num_neurons
        self.ds_graph = ds_graph

        self.num_layers = len(num_neurons)
        self.num_features = num_features
        self.learning_rate = learning_rate or 0.001
        self.adversarial = adversarial or False
        self.initialisation_params = initialisation_params or {}
        self.l2 = l2 or False
        self.l = l or 0.05

        #optional parameters
        self.optimizer = optimizer or tf.train.AdamOptimizer  #tf.train.GradientDescentOptimizer
        self.activations = activations or [tf.nn.relu  #tf.nn.tanh
                                           ] * self.num_layers  #tanh,relu, 
        self.initialisation_scheme = initialisation_scheme or tf.keras.initializers.he_normal  ##tf.contrib.layers.xavier_initializer  #
        #tf.contrib.layers.xavier_initializer  #tf.truncated_normal  #tf.random_uniform#
        self.num_epochs = num_epochs or meta_num_epochs
        self.seed = seed or None

        self.initialise_graph
        self.initialise_session
        print('initialising Network {}'.format(type(self)))
        
    

    @lazy_property
    def initialise_graph(self):
        #initialise graph
        self.g = tf.Graph()
        #build graph with self.graph as default so nodes get appended
        with self.g.as_default():
            dataset_graph = tf.import_graph_def(self.ds_graph, return_elements = ['X:0',"y:0"])
            self.next = self.g.get_tensor_by_name('import/next:0')
            self.init_network
            self.predict_graph
            self.error_graph
            self.train_graph
            self.init = tf.global_variables_initializer()

    @lazy_property
    def initialise_session(self):
        #initialise session
        self.session = tf.Session(graph=self.g)
        #initialise global variables
        self.session.run(self.init)

    @lazy_property
    def init_network(self):
        if self.seed:
            tf.set_random_seed(self.seed)
        #inputs
        self.X = self.next[0]
        self.y = self.next[1]
        #lists for storage
        self.w_list = []
        self.b_list = []

        #add input x first weights
        #        self.w_list.append(
        #            tf.Variable(
        #                self.initialisation_scheme(
        #                    [self.num_features, self.num_neurons[0]]),
        #                name='w_0'))  #first Matrix
        initialiser = self.initialisation_scheme(seed=self.seed,
                                                 **self.initialisation_params)
        self.w_list.append(
            tf.Variable(
                initialiser([self.num_features, self.num_neurons[0]]),
                name='w_0'))

        #for each layer over 0 add a n x m matrix and a bias term
        for i, num_neuron in enumerate(self.num_neurons[1:]):
            n_inputs = self.num_neurons[i]  #for first hidden layer 3
            n_outputs = self.num_neurons[i + 1]  #for first hidden layer 5

            self.w_list.append(
                tf.Variable(
                    initialiser([n_inputs, n_outputs]), name='w_' + str(i)))
            self.b_list.append(
                tf.Variable(tf.ones(shape=[n_inputs]), name='b_' + str(i)))

        #add last layer m  x 1 for output
        self.w_list.append(
            tf.Variable(initialiser([self.num_neurons[-1], 1]),
                        name='w_-1'))  #this is a regression
        self.b_list.append(
            tf.Variable(
                tf.ones(shape=[self.num_neurons[-1]]), name='b_' + str(
                    len(self.num_neurons) + 1)))

    @lazy_property
    def predict_graph(self):
        #set layer_input to input
        layer_input = self.X

        #for each layer do
        for i, w in enumerate(self.w_list):

            #z = input x Weights
            a = tf.matmul(layer_input, w, name='matmul_' + str(i))

            #z + bias
            if i < self.num_layers:
            #if i > 0:
              bias = self.b_list[i]
              a = tf.add(a, bias)

            #a = sigma(z) if not last layer and regression
            if i < self.num_layers:

                a = self.activations[i](a)
            #set layer input to a for next cycle
            layer_input = a

        return a

    @lazy_property
    def error_graph(self):

        #y_hat is a // output of prediction graph
        y_hat = self.predict_graph

        #error is mean squared error of placehilder y and prediction
        error = tf.losses.mean_squared_error(self.y, y_hat)  #tf.square(
        #self.y - y_hat)  #

        if self.l2:
            # Loss function with L2 Regularization with beta=0.01
            regularizers = tf.reduce_sum(
                [tf.nn.l2_loss(weights) for weights in self.w_list])
            error = tf.reduce_mean(error + self.l * regularizers)

        return error

    @lazy_property
    def train_graph(self):

        #error is the error from error graph
        error = self.error_graph

        #optimizer is self.optimizer
        optimizer = self.optimizer(learning_rate=self.learning_rate)

        return optimizer.minimize(error)

    def train_offline(self, X, y):
        for epoch in range(self.num_epochs):

            self.session.run(self.train_graph,
                             feed_dict={self.X: X,
                                        self.y: y})

    def fit(self, X, y, shuffle=True, online=True):
        #print('X is {}'.format(X[:10]))

        #if shuffle:

        if online:
            for epoch in range(self.num_epochs):
                if shuffle:
                    X, y, _ = self.shuffle_data(X, y)
                X = self.check_input_dimensions(X)
                y = self.check_input_dimensions(y)
                self.train_one_epoch(X, y, shuffle)
        else:
            if shuffle:
                X, y, _ = self.shuffle_data(X, y)
            X = self.check_input_dimensions(X)
            y = self.check_input_dimensions(y)
            self.train_offline(X, y)
            
            
    def fit(self,epochs):
      for epoch in range(epochs):
        self.session.run(self.train_graph)
        

    def train_one_epoch(self, epoch_X, epoch_y, shuffle=True):

        if shuffle == True:
            epoch_X, epoch_y, _ = self.shuffle_data(
                #np.squeeze(epoch_X), np.squeeze(epoch_y))
                epoch_X,
                epoch_y)
        #print(epoch_X[:10])
        epoch_X = self.check_input_dimensions(epoch_X)
        epoch_y = self.check_input_dimensions(epoch_y)
        for batch_X, batch_y in zip(epoch_X, epoch_y):
            self.train_one(batch_X, batch_y)
            if self.adversarial:
                self.train_one(batch_X + 0.05, batch_y)
                self.train_one(batch_X - 0.05, batch_y)

    def train_one(self, batch_X, batch_y):
        batch_X = self.check_input_dimensions(batch_X)
        batch_y = self.check_input_dimensions(batch_y)
        batch_X = np.array(batch_X).T

        batch_X, _, batch_y, _ = train_test_split(
            batch_X, batch_y, test_size=0.00, random_state=self.seed)

        #print('what')
        #print('X size: {}'.format(batch_X.shape))
        #print('y size: {}'.format(batch_y.shape))
        self.session.run(self.train_graph,
                         feed_dict={self.X: batch_X,
                                    self.y: batch_y})

    def train_and_evaluate(self, X, y, shuffle=False):
        errors = []
        for epoch in range(self.num_epochs):

            #self.session.run(self.train_graph,
            #                 feed_dict={self.X: X,
            #                            self.y: y})
            self.train_one_epoch(X, y, shuffle)

            errors += list(np.sqrt((y - self.predict(X))**2))

            #self.session.run(self.predict_graph,
            #                          feed_dict={self.X: X,
            #                                     self.y: y}))**2))

        return errors

    def predict(self, X):
        X = self.check_input_dimensions(X)

        return self.session.run(self.predict_graph,
                                feed_dict={self.X: X}).squeeze()

    def kill(self):
        self.session.close()

    def check_input_dimensions(self, array):
        """Makes sure arrays are compatible with Tensorflow input
        can't have array.shape = (X,),
        needs to be array.shape = (X,1)"""
        #y = array
        #y = np.reshape(y, [y.shape[0], 1])
        #return y
        if len(array.shape) <= 1:

            return np.expand_dims(array, 1)
        else:

            return array

    def shuffle_data(self, a, b):
        assert len(a) == len(b)
        p = np.random.permutation(len(a))
        sorted_index = np.argsort(p)
        p = np.squeeze(p)
        return a[p], b[p], sorted_index

    def network_mutli_dimensional_scatterplot(self, X_test, y_test, X=None,
                                              y=None, figsize=(20, 50),
                                              filename=None):

        #y_hat = self.predict(X_test)
        #print(pred_dict)
        #std = self.predict_var(X_test)
        y_hat, std = self.get_prediction_and_std(X_test)

        #plt.rcParams["figure.figsize"] = (20,20)
        fig = plt.figure(figsize=figsize)
        #plt.scatter(X[:,5],y)

        num_features = len(X_test.T)
        for i, feature in enumerate(X_test.T):
            #sort the arrays
            s = np.argsort(feature)
            var = y_hat[s] - std[s]
            var2 = y_hat[s] + std[s]
            print(feature.shape)
            print(var.shape)

            plt.subplot(num_features, 1, i + 1)
            plt.plot(
                feature[s],
                y_hat[s],
                label='predictive Mean', )
            plt.fill_between(feature[s].ravel(), y_hat[s].ravel(),
                             var.ravel(), alpha=.3, color='b',
                             label='uncertainty')
            plt.fill_between(feature[s].ravel(), y_hat[s].ravel(),
                             var2.ravel(), alpha=.3, color='b')
            plt.scatter(feature[s], y_test[s], label='data', s=20,
                        edgecolor="black", c="darkorange")
            plt.xlabel("data")
            plt.ylabel("target")
            plt.title("Ensemble")
            plt.legend()
            if filename is not None:
                plt.savefig(filename)

        if filename is not None:
            plt.savefig(filename)
        #plt.show()
        return fig

    def compute_rsme(self, X, y):
        y_hat = self.predict(X)
        #y_hat = pred_dict['means']
        #std = pred_dict['stds']        #print(y_hat.shape,std.shape,y.shape)

        return np.sqrt(np.mean((y_hat - y)**2))

    def score(self, X, y):
        return self.compute_rsme(X, y)

    def compute_error_vec(self, X, y):
        y_hat = self.predict(X)
        return y - y_hat

In [39]:

class EnsembleParent(object):
    """
    https://github.com/Hvass-Labs/TensorFlow-Tutorials/blob/master/05_Ensemble_Learning.ipynb
    """

    def __init__(self, estimator_stats=None, num_epochs=10):
        pass

    def train(self, X, y):
        pass

    def predict(self, X, return_samples=False):
        pass

    def plot(self, X, y, plot_samples=False, original_func=None,
             sorted_index=None):
        preds_dict = self.predict(X, True)
        y_hats = np.array([mean[0] for mean in preds_dict['means']])
        y_stds = np.array([std[0] for std in preds_dict['stds']])
        samples = preds_dict['samples']
        X_plot = [x[0] for x in np.array(X)]  #[sorted_index]]

        #print(y_hats.shape)
        #print(y_stds.shape)
        #print(y.shape)
        #print(np.array(X).shape)
        #X_plot = X_plot[sorted_index]
        y = np.array(y)  #[sorted_index]
        y_hats = y_hats  #[sorted_index]
        y_stds = y_stds  #[sorted_index]

        plt.plot(X, y, 'k*', label='Data', alpha=0.4)

        plt.plot(X_plot, y_hats, label='predictive mean')
        plt.fill_between(X_plot, y_hats + y_stds, y_hats, alpha=.3, color='b')
        plt.fill_between(X_plot, y_hats - y_stds, y_hats, alpha=.3, color='b')

        if plot_samples:
            for i, sample in enumerate(samples):

                plt.plot(X_plot, sample, label='sample {}'.format(i),
                         alpha=max(1 / len(samples), 0.3))
        if original_func:
            y_original = original_func(X_plot)
            plt.plot(X, y_original, label='generating model')
        plt.legend()


class VanillaEnsemble(EnsembleParent):
    """
    https://github.com/Hvass-Labs/TensorFlow-Tutorials/blob/master/05_Ensemble_Learning.ipynb
    """

    def __init__(self,ds_graph = None, estimator_stats=None, num_epochs=10):

        default_ensemble = [{
            'ds_graph' : gdef,
            'num_neurons': [10, 50, 20],
            'num_epochs': num_epochs,
            'seed':42
        }, {
            'ds_graph' : gdef,
            'num_neurons': [100, 10],
            'num_epochs': num_epochs,
            'seed': 43
        }, {
            'ds_graph' : gdef,
            'num_neurons': [5, 150],
            'num_epochs': num_epochs,
            'seed': 44
        }]

        self.estimator_stats = estimator_stats or default_ensemble
        self.estimator_list = [
            EnsembleNetwork(**x) for x in self.estimator_stats
        ]

    def fit(self, epochs = 10):
        '''This is where we build in the Online Bootstrap'''
        for estimator in self.estimator_list:
            #estimator.train_and_evaluate(X, y,shuffle=False)
            estimator.fit(epochs)#(X,y)
        #system('say training  complete')
        
    def train(self,X,y,epochs = 10):
      for epoch in epochs:
        for estimator in self.estimator_list:
          estimator.train(X,y)

    def predict(self, X, return_samples=False):
        pred_list = []
        for estimator in self.estimator_list:
            prediction = estimator.predict(X)
            pred_list.append(prediction)
        #for i,sample in enumerate(pred_list):
        #   assert(np.isnan(sample) is False), 'sample {} contains NaN'.format(i)
        stds = np.std(pred_list, 0)
        means = np.mean(pred_list, 0)
        #assert(np.isnan(stds) is False)
        #assert(np.isnan(means) is False)
        return_dict = {'stds': stds, 'means': means}
        if return_samples:
            return_dict['samples'] = pred_list
        #system('say prediction complete')

        return return_dict


class OnlineBootstrapEnsemble(VanillaEnsemble):
    """
    https://github.com/Hvass-Labs/TensorFlow-Tutorials/blob/master/05_Ensemble_Learning.ipynb
    """
    def __init__(self,gdef, estimator_stats = None,num_estimators=10,num_epochs=10,seed=10):
        
        default_ensemble = [{
            'ds_graph' : gdef,
            'num_neurons': [10, 5, 5, 2],
            'num_epochs': num_epochs,
            'seed':42
        },
            {
            'ds_graph' : gdef,
            'num_neurons': [10, 10, 5],
            'num_epochs': num_epochs,
            'seed': 43
        }, {
            'ds_graph' : gdef,
            'num_neurons': [5, 15, 5],
            'num_epochs': num_epochs,
            'seed': 44
        }
        ]

        self.estimator_stats = estimator_stats or default_ensemble
        self.estimator_list = [
            EnsembleNetwork(**x) for x in self.estimator_stats
        ]
        

    def fit(self, epochs = 10):
      for i in range(epochs*2): # only training in half of the cases
        for estimator in self.estimator_list:
          if np.random.random() > 0.5:
            #estimator.train_and_evaluate(X, y,shuffle=False)
              estimator.fit(epochs)#(X,y)
        #system('say training  complete')

    def train(self,X,y,epochs = 10):
      for epoch in range(epochs*2):
        
        for estimator in self.estimator_list:
          #mask = np.random.choice([0,1], size= len(y)).astype(bool)
          #X = X[mask]
          #y = y[mask]
          estimator.train(X,y)

            
    def predict(self,X,return_samples=False):
        pred_list = []
        for estimator in self.estimator_list:
            prediction = estimator.predict(X)
            pred_list.append(prediction)
            
        stds = np.std(pred_list,0)
        means = np.mean(pred_list,0)
        
        
        return_dict = {'stds':stds,'means':means}
        if return_samples:
            return_dict['samples'] = pred_list
        #system('say prediction complete')

        return return_dict
            


In [40]:
meta_epochs = 20
plt.figure(figsize = (16,8*meta_epochs))

<Figure size 1152x11520 with 0 Axes>

<Figure size 1152x11520 with 0 Axes>

In [41]:
#X

In [42]:
# nn = EnsembleNetwork(gdef, learning_rate= 0.01)

In [43]:
# nn.train(X,y)

In [44]:
# plt.plot(nn.predict(X))

In [45]:
# for i in range(100):nn.train(X,y)

In [46]:
# plt.plot(nn.predict(X))

In [47]:
seeds = [1,2,3,4,5]
layers = [
    [10, 50, 20, 30, 10],
    [100, 50],
    [30, 30, 30],
    [5, 10, 100],
    [50, 50, 50]
]

In [48]:
estimator_stats = [{
            'ds_graph' : gdef,
            'num_neurons': num_neurons,
            'num_epochs': 10,
            'seed':seed,
    'activations': [tf.nn.tanh,tf.nn.tanh,tf.nn.tanh,tf.nn.tanh,tf.nn.relu]
    #activagtions:tf.nn.relu  #tf.nn.tanh
        } for seed, num_neurons in zip(seeds,layers)]

In [ ]:
van = OnlineBootstrapEnsemble(gdef,num_epochs=10,estimator_stats=estimator_stats)
#van = VanillaEnsemble(gdef,num_epochs=10,estimator_stats=estimator_stats)






In [ ]:
def get_mean_var(pred_dict):
    return pred_dict['means'],pred_dict['stds']

In [ ]:
len(van.estimator_list)

In [ ]:
mean, var = get_mean_var(van.predict(X))

plt.plot(X,mean,label='X')
plt.plot(X,y,'r*')
plt.fill_between(np.squeeze(X),np.squeeze(mean), np.squeeze(mean+var),alpha=0.3, color='b')
plt.fill_between(np.squeeze(X),np.squeeze(mean), np.squeeze(mean-var),alpha=0.3, color='b')
plt.legend()

In [ ]:
#van.fit(1000)

In [ ]:
# mean, var = get_mean_var(van.predict(X))

# plt.plot(X,mean,label='X')
# plt.plot(X,y,'r*')
# plt.fill_between(np.squeeze(X),np.squeeze(mean), np.squeeze(mean+var),alpha=0.3, color='b')
# plt.fill_between(np.squeeze(X),np.squeeze(mean), np.squeeze(mean-var),alpha=0.3, color='b')
# plt.legend()

In [ ]:
# a = van.fit(100)

In [ ]:
import tqdm

In [ ]:
import numpy as np

import scipy 


def rmse(y_hat, y):
    return np.sqrt(np.mean((y_hat - y)**2))


def coverage_probability(prediction, variance, truth):
    CP = 0
    for y_hat, s, y in zip(prediction, variance, truth):
        if y_hat + s > y > y_hat - s:
            CP += 1

    #if (y - 2 * s < 0) and (y + 2 * s > 0):
    #    CP += 1
    return CP / len(truth)

def error_uncertainty_correlation(prediction,variance,truth):
    prediction, variance, truth = prediction.flatten(), variance.flatten(), truth.flatten()
    assert prediction.shape == truth.shape == variance.shape, 'shapes are: pred {} truth {} variance {}'.format(prediction.shape, truth.shape, variance.shape)
    error = (prediction - truth.flatten())**2
    correlation = scipy.stats.pearsonr(error.flatten(),variance.flatten())
    
    #np.correlate(error.flatten(),variance.flatten())
    return correlation

In [ ]:
error_list = list()
coverage_list = list()
cobeau_list = list()

for i in tqdm.tqdm(range(meta_epochs)):


  mean, var = get_mean_var(van.predict(X))
  plt.figure(figsize = (16,8*10))

  plt.subplot(meta_epochs,1,i+1)
  plt.plot(X,mean,label='X')
  plt.plot(X,y,'r*')
  plt.fill_between(np.squeeze(X),np.squeeze(mean), np.squeeze(mean+var),alpha=0.3, color='b')
  plt.fill_between(np.squeeze(X),np.squeeze(mean), np.squeeze(mean-var),alpha=0.3, color='b')
  plt.legend()


  error = rmse(mean,y)
  coverage = coverage_probability(mean,var,y)
  cobeau = error_uncertainty_correlation(mean,var,y)
    
  error_list.append(error)
  coverage_list.append(coverage)
  cobeau_list.append(cobeau)

  print('rsme is {}. \n Coverage is {}. \n COBEAU is {}.'.format(error,coverage,cobeau))


  

  plt.show()
  a = van.fit(25)

plt.figure()
plt.plot(error_list,label='error')
plt.plot(coverage_list,label='coverage')
plt.plot(cobeau_list,label='cobeau')

In [ ]:
van.plot(X,y)

Bootstrap-ish


In [ ]:
with tf.Graph().as_default() as g_1:
  dataset = tf.data.Dataset.from_tensor_slices((X, y))
  dataset = dataset.repeat()
  dataset = dataset.shuffle(100)
  dataset = dataset.batch(100)
  it = dataset.make_one_shot_iterator()
  get_next = tf.identity(it.get_next(), name = 'next')
  X_ =  tf.identity(it.get_next()[0], name="X")
  y_ =  tf.identity(it.get_next()[1], name="y")

  gdef = g_1.as_graph_def()
  
  #g_1.get_tensor_by_name('X:0')
  #g_1.get_tensor_by_name("y:0")

  #it = dataset.make_one_shot_iterator()

#X,y = it.get_next()

In [ ]:
plt.plot(nn.predict(X[10:50]))
plt.plot(y[10:50])
#plt.plot(nn.predict_meh())

In [ ]:
tf.reset_default_graph()


model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(12, input_dim=1, kernel_initializer='normal', activation='tanh'))
model.add(tf.keras.layers.Dense(8, activation='tanh'))
model.add(tf.keras.layers.Dense(1, activation='linear'))


model.compile(loss='mse', optimizer='adam', )#metrics=['mse','mae'])


model.summary()

In [ ]:
model.fit(dataset, epochs=1000, steps_per_epoch=10)

In [ ]:
plt.plot(model.predict(X)[:100])
plt.plot(y[:100])

In [ ]:
tf.reset_default_graph()


import numpy as np
import tensorflow as tf
keras = tf.keras

# tf.data.Dataset instance
tr_data = np.random.random((1000, 32)).astype(np.float32)
tr_label = np.random.randint(low=0, high=10, size = 1000).astype(np.int32)
tr_dataset = tf.data.Dataset.from_tensor_slices((tr_data, tr_label))
tr_dataset = tr_dataset.batch(batch_size=32)
tr_dataset = tr_dataset.repeat()

val_data = np.random.random((100, 32)).astype(np.float32)
val_label = np.random.randint(low=0, high=10, size = 100).astype(np.int32)
val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_label))
val_dataset = val_dataset.batch(batch_size=100).repeat()

# Training
model = keras.Sequential()
model.add(keras.layers.Dense(units=64, activation='relu'))
model.add(keras.layers.Dense(units=64, activation='relu'))
model.add(keras.layers.Dense(units=10, activation='softmax'))
model.compile(optimizer=tf.train.GradientDescentOptimizer(.01), 
              loss=keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

model.fit(tr_dataset, epochs = 5, steps_per_epoch = 1000 // 32,
          validation_data = val_dataset, validation_steps = 1)

In [ ]:
# with Session
tf.reset_default_graph()

with tf.Session() as sess:
  sess.run(predict)

In [ ]:
tf.reset_default_graph()

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((X, y))
dataset = dataset.repeat()
dataset = dataset.shuffle(1)
dataset = dataset.batch(10)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(32))
model.add(tf.keras.layers.Dense(32))
model.compile(optimizer='adam', loss='mean_squared_error')
# This builds the model for the first time:


In [ ]:
model.fit(dataset.make_one_shot_iterator().get_next()[0],dataset.make_one_shot_iterator().get_next()[1], epochs=5, steps_per_epoch=10)

In [ ]:
nn.fit_dataset(X,y)

In [ ]:
import tensorflow as tf
import numpy as np

class Network:
    def __init__(self, x_in, input_size):
        self.input_size = input_size
        # self.x_in = tf.placeholder(dtype=tf.float32, shape=(None, self.input_size))  # Original
        self.x_in = x_in
        self.output_size = 3

        #tf.reset_default_graph()  # This turned out to be the problem

        self.layer = tf.layers.dense(self.x_in, self.output_size, activation=tf.nn.relu)
        self.loss = tf.reduce_sum(tf.square(self.layer - tf.constant(0, dtype=tf.float32, shape=[self.output_size])))

data_array = np.random.standard_normal([4, 10]).astype(np.float32)
dataset = tf.data.Dataset.from_tensor_slices(data_array).batch(2)

model = Network(x_in=dataset.make_one_shot_iterator().get_next(), input_size=dataset.output_shapes[-1])

NEW


In [ ]:
import numpy as np
import tensorflow as tf
from sklearn import datasets

# 推論

def inference(x):
    with tf.name_scope('l1') as scope:
        w_l1 = tf.Variable(tf.truncated_normal(
            [10, 256], stddev=0.33), name="w_l1")
        b_l1 = tf.Variable(tf.constant(1.0, shape=[256]), name="b_l1")
        h_l1 = tf.nn.relu(tf.matmul(x, w_l1) + b_l1)

    with tf.name_scope('l2') as scope:
        w_l2 = tf.Variable(tf.truncated_normal(
            [256, 1], stddev=0.33), name="w_l2")
        b_l2 = tf.Variable(tf.constant(1.0, shape=[1]), name="b_l2")
        output = tf.nn.relu(tf.matmul(h_l1, w_l2) + b_l2)

    return output

# 損失関数
def loss(model, y):
    return tf.reduce_mean(tf.square(model - y), name="loss")

# 学習
def training(loss, rate):
    return tf.train.AdagradOptimizer(rate).minimize(loss)

def main():
    diabetes = datasets.load_diabetes()
    data = diabetes["data"].astype(np.float32)
    target = diabetes['target'].astype(
        np.float32).reshape(len(diabetes['target']), 1)

    MAX_SIZE = data.shape[0]
    TEST_N = 100
    N = MAX_SIZE - TEST_N
    BATCH_SIZE = 10
    MAX_STEPS = 300

    train_x, test_x = np.vsplit(data, [N])
    train_y, test_y = np.vsplit(target, [N])

    # symbolic variables
    x = tf.placeholder(tf.float32, shape=[None, 10])
    y = tf.placeholder(tf.float32, shape=[None, 1])

    model = inference(x)
    loss_value = loss(model, y)
    train_op = training(loss_value, 0.04)

    best = float("inf")
    init_op = tf.global_variables_initializer()

    with tf.Session() as sess:
        sess.run(init_op)
        for step in range(MAX_STEPS + 1):
            for i in range(N // BATCH_SIZE):
                batch = BATCH_SIZE * i
                train_batch_x = train_x[batch:batch + BATCH_SIZE]
                train_batch_y = train_y[batch:batch + BATCH_SIZE]

                loss_train = sess.run(loss_value, feed_dict={
                                      x: train_batch_x, y: train_batch_y})
                sess.run(train_op, feed_dict={
                         x: train_batch_x, y: train_batch_y})

            if loss_train < best:
                best = loss_train
                best_match = sess.run(model, feed_dict={x: test_x, y: test_y})

            if step % 10 == 0:
                cor = np.corrcoef(best_match.flatten(), test_y.flatten())
                print('step : {}, train loss : {}, test cor : {}'.format(
                    step, best, cor[0][1]))

In [ ]:
main()